In [ ]:
!dwave auth login # To login into the dwave leap account

Usage: dwave auth login [OPTIONS]
Try 'dwave auth login --help' for help.

Error: Got unexpected extra arguments (# To login into the dwave leap account)


: 

In [ ]:
!dwave config create # To configure your account with solver

Usage: dwave config create [OPTIONS]
Try 'dwave config create --help' for help.

Error: Got unexpected extra arguments (# To configure your account with solver)


: 

In [ ]:
!dwave ping --client qpu # To call your qpu availibilty from Dwave-leap account

Usage: dwave ping [OPTIONS]
Try 'dwave ping --help' for help.

Error: Got unexpected extra arguments (# To call your qpu availibilty from Dwave-leap account)


: 

In [15]:
from docplex.mp.model import Model
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from qiskit_optimization.converters import QuadraticProgramToQubo
# from qiskit import BasicAer
# from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import CplexOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty
from qiskit_optimization.applications import BinPacking, Tsp
from brute_force_bpp import *


In [5]:
def min_bins(weights, C):# n is the number of bins
    n  = len(weights)
    w_s = sum(weights)
    if w_s > n * C:
        print('increase capacity or the number of bins')
        return
    while w_s < n * C :
        n -= 1
        if w_s > n * C :
            return n + 1
        elif w_s == n * C:
            return n
        else:
            continue
weights = [3, 2, 3, 2]  # Weights of 4 items
C = 5
min_bins(weights, C)


def data_bins(result, weights, n, m, C, simplify=False):
    """
    Organizes the results of the ILP Bin Packing optimization into bins.

    Parameters:
    result (OptimizationResult): Optimization result from the solver.
    weights (list): Weights of each item.
    n (int): Number of items.
    m (int): Maximum possible number of bins.
    C (int): Capacity of each bin.
    simplify (bool): Flag to simplify the output (optional).

    Returns:
    dict: Dictionary with keys "bins", "items", and "index" for bin data.
    """
    # Extract variables dictionary from the result
    variables_dict = result.variables_dict

    # Initialize arrays for decision variables
    x_vars = np.zeros((m, n))
    y_vars = np.zeros(m)

    # Retrieve values from the variables dictionary
    for j in range(m):
        y_vars[j] = variables_dict.get(f"y_{j}", 0)  # Bin j status
        for i in range(n):
            x_vars[j, i] = variables_dict.get(f"x_{i}_{j}", 0)  # Item i in bin j

    # Calculate the weight of items in each bin
    items = x_vars * weights
    bins = y_vars

    return {"bins": bins, "items": items, "index": np.arange(m)}

def plot_bins(result, weights, n, m, C, simplify=False):
    """
    Plots the results of the ILP Bin Packing solution in a bar diagram.

    Parameters:
    result (OptimizationResult): Optimization result from the solver.
    weights (list): Weights of each item.
    n (int): Number of items.
    m (int): Maximum possible number of bins.
    C (int): Capacity of each bin.
    simplify (bool): Flag to simplify the plot (optional).
    """
    res = data_bins(result, weights, n, m, C, simplify)
    plt.figure(figsize=(10, 6))

    # Bin index and initial bar for first item
    ind = res["index"]
    plt.bar(ind, res["items"][:, 0], label=f"item 0")
    suma = res["items"][:, 0]  # Initialize bottom for stacking bars

    # Stack items in each bin
    for j in range(1, n):
        plt.bar(ind, res["items"][:, j], bottom=suma, label=f"item {j}")
        suma += res["items"][:, j]

    # Add a line for the bin capacity
    plt.hlines(C, -0.5, m - 0.5, linestyle="--", color="r", label="Max Capacity (C)")

    plt.xticks(ind)
    plt.xlabel("Bin")
    plt.ylabel("Weight")
    plt.legend()
    plt.title("Bin Packing Solution")
    plt.show()


## Formulation

The **Bin Packing Problem (BPP)** is a classic optimization problem where we are given a set of items with specific weights and bins with a fixed capacity. The goal is to pack all items into the minimum number of bins without exceeding the capacity of any bin.

### Problem Definition

- Given $n$ items with weights $w_1, w_2, \ldots, w_n$ and bin capacity $C$.
- Minimize the number of bins used.

### ILP Formulation

**Decision Variables:**
- $x_{ij}$: 1 if item $i$ is in bin $j$, 0 otherwise.
- $y_j$: 1 if bin $j$ is used, 0 otherwise.

**Objective Function:**
$$
\text{Minimize } \sum_{j=1}^{m} y_j
$$
where $m$ is an upper bound on the number of bins.

**Constraints:**
1. Each item must be assigned to exactly one bin:
   $$
   \sum_{j=1}^{m} x_{ij} = 1, \quad \forall i \in \{1, 2, \ldots, n\}
   $$
2. The total weight in each bin cannot exceed its capacity:
   $$
   \sum_{i=1}^{n} w_i x_{ij} \leq C y_j, \quad \forall j \in \{1, 2, \ldots, m\}
   $$
3. Binary constraints:
   $$
   x_{ij} \in \{0, 1\}, \quad y_j \in \{0, 1\}
   $$

### Explanation of Terms
- **Objective Function**: Represents the total number of bins used.
- **Constraint 1**: Ensures each item is placed in exactly one bin.
- **Constraint 2**: Ensures the weight in each bin does not exceed $C$.
- **Binary Constraints**: Ensure $x_{ij}$ and $y_j$ are binary variables.


In [6]:
# Input Data

np.random.seed(2)
n = 2 # number of bins
C = 20 # max weight of a bin

weights = np.random.randint(1,C,n) # Randomly picking the item weight
def bpp_to_ilp(weights, C):
    # Initialize model
    model = Model(name="Bin Packing Problem")
    m = min_bins(weights, C)
    # Decision Variables
    # x[i, j] = 1 if item i is placed in bin j, 0 otherwise
    x = {(i, j): model.binary_var(name=f"x_{i}_{j}") for i in range(n) for j in range(m)}

    # y[j] = 1 if bin j is used, 0 otherwise
    y = {j: model.binary_var(name=f"y_{j}") for j in range(m)}
    # Constraints

    # 1. Each item must be assigned to exactly one bin
    for i in range(n):
        model.add_constraint(model.sum(x[i, j] for j in range(m)) == 1, ctname=f"assign_item_{i}")

    # 2. Total weight in each bin cannot exceed bin capacity
    for j in range(m):
        model.add_constraint(model.sum(weights[i] * x[i, j] for i in range(n)) <= C * y[j], ctname=f"capacity_bin_{j}")

    # Objective: Minimize the number of bins used
    model.minimize(model.sum(y[j] for j in range(m)))
    
    return model


In [7]:
weights = [2,3,1,3]
C = 5
mod = bpp_to_ilp(weights, C)

In [ ]:
mod = from_docplex_mp(model)
# Solving Quadratic Program using CPLEX
cplex = CplexOptimizer()
result = cplex.solve(mod)
print(result)
plot_bins(result, weights, n, m, C)
result.variables_dict


MissingOptionalLibraryError: "The 'CPLEX' library is required to use 'CplexOptimizer'. You can install it with 'pip install 'qiskit-optimization[cplex]''."

: 

### Objective Function

Minimize:

$$
\sum_{i=1}^{m} y_i + \alpha \left[ - \sum_{i=1}^{m} \left( C y_i - \sum_{j=1}^{n} w_j x_{ij} \right) + \left( \sum_{i=1}^{m} \left( C y_i - \sum_{j=1}^{n} w_j x_{ij} \right) \right)^2 \right]
$$

Where:
- $y_i$ is a binary variable indicating whether bin $i$ is used (1) or not (0).
- $x_{ij}$ is a binary variable indicating whether item $j$ is assigned to bin $i$.

### Subject to:

#### 1. Assignment Constraint:
Each item must be assigned to exactly one bin:

$$
\sum_{i=1}^{m} x_{ij} = 1 \quad \forall j = 1, \dots, n
$$

#### 2. Capacity Constraint:
The total weight in each bin cannot exceed its capacity if the bin is used:

$$
\sum_{j=1}^{n} w_j x_{ij} \leq C y_i \quad \forall i = 1, \dots, m
$$

#### 3. Binary Constraints:

For the assignment variables:

$$
x_{ij} \in \{0, 1\} \quad \forall i = 1, \dots, m \quad \text{and} \quad j = 1, \dots, n
$$

For the bin usage variables:

$$
y_i \in \{0, 1\} \quad \forall i = 1, \dots, m
$$


In [8]:
def ilp_to_qubo(docplex_model):
    """
    Converts a given ILP Docplex model to a QUBO model.

    Parameters:
    docplex_model: Model
        A Docplex model representing an ILP problem.

    Returns:
    qubo: Quadratic """

    qp = from_docplex_mp(docplex_model)
    qp2qubo = QuadraticProgramToQubo()
    qubo = qp2qubo.convert(qp)
    # print(qubo.prettyprint())
    return qubo


In [9]:
qresult = ilp_to_qubo(mod)

qresult.get_num_binary_vars()

12

In [16]:
from dwave.system import DWaveSampler, EmbeddingComposite
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

def solve_qubo_on_dwave(qubo):
    """
    Solves the QUBO on D-Wave's quantum annealer.

    Parameters:
    qubo: QuadraticProgram
        The QUBO problem generated by ilp_to_qubo.

    Returns:
    result: dict
        The solution from D-Wave's quantum annealer.
    """
    # Convert the QUBO to a format compatible with D-Wave
    qubo_dict = qubo.to_ising()[0]

    # Define the sampler with an embedding to handle larger problems
    sampler = EmbeddingComposite(DWaveSampler())

    # Run the sampler on the QUBO model
    sampleset = sampler.sample_qubo(qubo_dict, num_reads=100)

    # Extract the best solution
    best_solution = sampleset.first.sample
    best_energy = sampleset.first.energy

    print("Best Solution:", best_solution)
    print("Energy:", best_energy)

    return best_solution, best_energy

# Example usage:
# docplex_model = # Define your ILP model using docplex here
# qubo_model = ilp_to_qubo(docplex_model)
# best_solution, best_energy = solve_qubo_on_dwave(qubo_model)

solve_qubo_on_dwave(qresult)

ValueError: API token not defined

In [ ]:
result1 = cplex.solve(qresult)
print(result1)


: 

In [ ]:
from brute_force_bpp import *

weights = [3, 2, 3, 2, 1]  # Weights of 4 items
C =  max(weights) # Bin capacity
brute_force(weights, C)



NameError: name 'create_y' is not defined


IPython -- An enhanced Interactive Python

IPython offers a fully compatible replacement for the standard Python
interpreter, with convenient shell features, special commands, command
history mechanism and output results caching.

At your system command line, type 'ipython -h' to see the command line
options available. This document only describes interactive features.

GETTING HELP
------------

Within IPython you have various way to access help:

  ?         -> Introduction and overview of IPython's features (this screen).
  object?   -> Details about 'object'.
  object??  -> More detailed, verbose information about 'object'.
  %quickref -> Quick reference of all IPython specific syntax and magics.
  help      -> Access Python's own help system.

If you are in terminal IPython you can quit this screen by pressing `q`.


MAIN FEATURES
-------------

* Access to the standard Python help with object docstrings and the Python
  manuals. Simply type 'help' (no quotes) to invoke it.

* Ma

In [36]:
weights = [3,2,1,2]  # Weights of 4 items
C = 5  # Bin capacity
# bpp = BinPacking(weights,C,len(weights))
# qp = bpp.to_quadratic_program()

# qp2qubo = QuadraticProgramToQubo()


# print("Ising Hamiltonian:")
# # print(str(qubitOp))
# # # H_matrix = qubitOp.to_matrix()
# print(qp.get_num_vars())

# my model 
mod1 = bpp_to_ilp(weights, C)
mod1 = ilp_to_qubo(mod1)

print(mod1.get_num_binary_vars())

# qubo = qp2qubo.convert(mod1)
qubitOp, offset = mod1.to_ising()
qubitOp


12


SparsePauliOp(['IIIIIIIZIIII', 'IIIIIIZIIIII', 'IIIIIIIIIIIZ', 'IIIIIIIIIIZI', 'IIIIIIIIIZII', 'IIIIIIIIZIII', 'IIIIIIIIIIZZ', 'IIIIIIIIIZIZ', 'IIIIIIIZIIIZ', 'IIIIIZIIIIIZ', 'IIIIIZIIIIII', 'IIIIZIIIIIIZ', 'IIIIZIIIIIII', 'IIIZIIIIIIIZ', 'IIIZIIIIIIII', 'IIIIIIIIZIZI', 'IIIIIIZIIIZI', 'IIZIIIIIIIZI', 'IIZIIIIIIIII', 'IZIIIIIIIIZI', 'IZIIIIIIIIII', 'ZIIIIIIIIIZI', 'ZIIIIIIIIIII', 'IIIIIIIIZZII', 'IIIIIIIZIZII', 'IIIIIZIIIZII', 'IIIIZIIIIZII', 'IIIZIIIIIZII', 'IIIIIIZIZIII', 'IIZIIIIIZIII', 'IZIIIIIIZIII', 'ZIIIIIIIZIII', 'IIIIIZIZIIII', 'IIIIZIIZIIII', 'IIIZIIIZIIII', 'IIZIIIZIIIII', 'IZIIIIZIIIII', 'ZIIIIIZIIIII', 'IIIIZZIIIIII', 'IIIZIZIIIIII', 'IIIZZIIIIIII', 'IZZIIIIIIIII', 'ZIZIIIIIIIII', 'ZZIIIIIIIIII'],
              coeffs=[ 37. +0.j,  37. +0.j, -22.5+0.j, -22.5+0.j, -15. +0.j, -15. +0.j,
   1.5+0.j,   9. +0.j, -22.5+0.j,   4.5+0.j,  -7.5+0.j,   9. +0.j,
 -15. +0.j,   9. +0.j, -15. +0.j,   9. +0.j, -22.5+0.j,   4.5+0.j,
  -7.5+0.j,   9. +0.j, -15. +0.j,   9. +0.j, -15. +0.j,   

In [37]:
num_wires = qubitOp.num_qubits
wires = list(range(num_wires))
# H_matrix = qubitOp.to_matrix()
num_wires
# H = qml.Hermitian(H_matrix, wires=wires)
# qp.prettyprint()
# print("Offset:", offset)

# min(qml.eigvals(H))



12

### VQA Approach

In [38]:
# solving Quadratic Program using exact classical eigensolver
from qiskit_algorithms import SamplingVQE, NumPyMinimumEigensolver

ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)


result.eigenstate
# def VQA_qubo(qubo_model):

#     qubo_model.to_ising()



Statevector([ 8.39142218e-16+1.09982316e-16j,
             -2.27316944e-17-2.43664870e-17j,
              3.96339546e-18-6.42110374e-18j, ...,
             -1.05224114e-17+1.06511455e-17j,
              5.01183412e-19+3.23428129e-18j,
             -2.36915645e-19+1.24427360e-19j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [29]:
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_algorithms import SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [30]:
algorithm_globals.random_seed = 123
seed = 10598

In [37]:
# construct SamplingVQE
optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

# run SamplingVQE
result = vqe.compute_minimum_eigenvalue(qubitOp)

# print results
x = BinPacking.sample_most_likely(result.eigenstate)
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))


energy: -97.22442844762084
time: 134.03024411201477
max-cut objective: 14.775571552379162
solution: [1 1 1 1 0 1 1 1 0 1 0 0]


ValueError: dimension mismatch